# MPIA Arxiv on Deck 2: Debugging notebook

In this notebook, I keep some first order commands for diagnostic of issues with papers.
Main definitions are taken from the main notebook.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

# Some useful definitions.
class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

We get the author list from the MPIA website

In [2]:
# Getting the list of authors can take sometimes (internet connection)
# Caching the MPIA author list to avoid running this line every time we restart the kernel.
import yaml
try:
    with open('tmp_mpia_authors.yml', 'r') as fin:
        mpia_authors = yaml.load(fin, yaml.BaseLoader)
    print("`mpia.get_mpia_mitarbeiter_list()`: restored from cache")
except FileNotFoundError:
    print("`mpia.get_mpia_mitarbeiter_list()`: cannot be restored from cache.")
    # get list from MPIA website
    # it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
    mpia_authors = mpia.get_mpia_mitarbeiter_list()
    with open('tmp_mpia_authors.yml', 'w') as fout:
        fout.write(yaml.dump(mpia_authors))

`mpia.get_mpia_mitarbeiter_list()`: restored from cache


In [3]:
# Get family names only
data = [k[-1] for k in mpia_authors]

# clean -- Having titles is something new at MPIA
data = [k.replace('Dr. ', '').strip() for k in data]

filtered_data = list(filter(mpia.filter_non_scientists, data))

name_variations = filter(lambda x: x is not None,
                         [mpia.consider_variations(name) for name in filtered_data])
mitarbeiter_list = sorted(filtered_data + list(name_variations))

lst = [(mpia.get_special_corrections(mpia.get_initials(name)), name) for name in mitarbeiter_list]
lst = [(mpia.family_name_from_initials(k[0]), k[0], k[1]) for k in lst]
lst = sorted(lst, key=lambda x: x[0])
lst = set(lst)

We get the paper to debug

In [4]:
which = "2303.00044"
paper = get_paper_from_identifier(which)
paper


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2303.00044-b31b1b.svg)](https://arxiv.org/abs/2303.00044) | **The Initial Mass Function and Other Stellar Properties Across the Core of the Hydra I Cluster**  |
|| Ilaria Lonoce, Wendy Freedman, Anja Feldmeier-Krause |
|*Appeared on*| *2023-02-28*|
|*Comments*| *32 pages, 18 figures. Accepted for publication in ApJ*|
|**Abstract**| The Hydra I cluster offers an excellent opportunity to study and compare therelic old stellar populations in the core of its two brightest galaxies. Inaddition, the differing kinematics of the two galaxies allows a test of thelocal validity of general scaling relations. In this work we present a directcomparison employing full spectral fitting of new high-quality long-slitoptical and NIR spectroscopic data. We retrieve age, metallicity and 19elemental abundances out to about 12 kpc within each galaxy, as well as the IMFin their central regions. Our results suggest that the inner 5 kpc region ofboth galaxies, despite their different masses, formed at the same time andevolved with a similar star formation time-scale and chemical enrichment,confirming their early formation in the cluster build up. Only the overallmetallicity and IMF radial profiles show differences connected with theirdifferent velocity dispersion profiles. The radial trend of the IMF positivelycorrelates with both [Z/H] and velocity dispersion. While the trends of the IMFwith metallicity agree with a global trend for both galaxies, the trends withthe velocity dispersion exhibit differences. The outer regions show signs ofmixed stellar populations with large differences in chemical content comparedto the centers, but with similar old ages.|

In [5]:
# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in lst]

hl_authors = highlight_authors_in_list(paper['authors'], hl_list, verbose=True)
matches = [(hl, orig) for hl, orig in zip(hl_authors, paper['authors']) if 'mark' in hl]
if not matches:
    warnings.warn(AffiliationWarning("WARNING: This paper does not seem to have MPIA authors."))
paper['authors'] = hl_authors
paper


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2303.00044-b31b1b.svg)](https://arxiv.org/abs/2303.00044) | **The Initial Mass Function and Other Stellar Properties Across the Core of the Hydra I Cluster**  |
|| Ilaria Lonoce, Wendy Freedman, <mark>Anja Feldmeier-Krause</mark> |
|*Appeared on*| *2023-02-28*|
|*Comments*| *32 pages, 18 figures. Accepted for publication in ApJ*|
|**Abstract**| The Hydra I cluster offers an excellent opportunity to study and compare therelic old stellar populations in the core of its two brightest galaxies. Inaddition, the differing kinematics of the two galaxies allows a test of thelocal validity of general scaling relations. In this work we present a directcomparison employing full spectral fitting of new high-quality long-slitoptical and NIR spectroscopic data. We retrieve age, metallicity and 19elemental abundances out to about 12 kpc within each galaxy, as well as the IMFin their central regions. Our results suggest that the inner 5 kpc region ofboth galaxies, despite their different masses, formed at the same time andevolved with a similar star formation time-scale and chemical enrichment,confirming their early formation in the cluster build up. Only the overallmetallicity and IMF radial profiles show differences connected with theirdifferent velocity dispersion profiles. The radial trend of the IMF positivelycorrelates with both [Z/H] and velocity dispersion. While the trends of the IMFwith metallicity agree with a global trend for both galaxies, the trends withthe velocity dispersion exhibit differences. The outer regions show signs ofmixed stellar populations with large differences in chemical content comparedto the centers, but with similar old ages.|

We get the (TeX) source
* retrieve the tarball
* find the main tex file and parse it
* parse for affiliations (but debugging so we do not stop if not found)
* generate the the output markdown

In [6]:
paper_id = f'{which:s}'
folder = f'tmp_{paper_id:s}'

if not os.path.isdir(folder):
    folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')

try:
    doc = latex.LatexDocument(folder, validation=validation)    
except AffiliationError as affilerror:
    msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
    print(msg)

# Hack because sometimes author parsing does not work well
if (len(doc.authors) != len(paper['authors'])):
    doc._authors = paper['authors']
if (doc.abstract) in (None, ''):
    doc._abstract = paper['abstract']

doc.comment = get_markdown_badge(paper_id) + " _" + paper['comments'] + "_"
doc.highlight_authors_in_list(hl_list, verbose=True)

full_md = doc.generate_markdown_text()

# replace citations
try:
    bibdata = latex_bib.LatexBib.from_doc(doc)
    full_md = latex_bib.replace_citations(full_md, bibdata)
except Exception as e:
    raise e

Found 93 bibliographic references in tmp_2303.00044/main.bbl.


In [7]:
Markdown(full_md)

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand{\afk}[1]{\textcolor{purple}{\uline{#1}}}$
$\newcommand{\afks}[1]{\textcolor{purple}{\sout{#1}}}$
$\newcommand{\il}[1]{\textcolor{cyan}{\uline{#1}}}$
$\newcommand{\ils}[1]{\textcolor{cyan}{\sout{#1}}}$
$\newcommand{\wf}[1]{\textcolor{blue}{\uline{#1}}}$
$\newcommand{\wfs}[1]{\textcolor{blue}{\sout{#1}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand{\afk}[1]{\textcolor{purple}{\uline{#1}}}$
$\newcommand{\afks}[1]{\textcolor{purple}{\sout{#1}}}$
$\newcommand{\il}[1]{\textcolor{cyan}{\uline{#1}}}$
$\newcommand{\ils}[1]{\textcolor{cyan}{\sout{#1}}}$
$\newcommand{\wf}[1]{\textcolor{blue}{\uline{#1}}}$
$\newcommand{\wfs}[1]{\textcolor{blue}{\sout{#1}}}$</div>



<div id="title">

# The initial mass function and other stellar properties across the core of the Hydra I cluster$\footnote{This paper includes data gathered with the 6.5 meterMagellan Telescopes located at Las Campanas Observatory, Chile.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.00044-b31b1b.svg)](https://arxiv.org/abs/2303.00044) _32 pages, 18 figures. Accepted for publication in ApJ_

</div>
<div id="authors">

Ilaria Lonoce, W. L. Freedman, <mark>A. Feldmeier-Krause</mark>

</div>
<div id="abstract">

**Abstract:** The Hydra I cluster offers an excellent opportunity to study and compare the relic old stellar populations in the core of its two brightest galaxies. In addition, the differing kinematics of the two galaxies allows a test of the local validity of general scaling relations. In this work we present a direct comparison employing full spectral fitting of new high-quality long-slit optical and NIR spectroscopic  data. We retrieve age, metallicity and 19 elemental abundances out to $\sim12$ kpc within each galaxy, as well as the IMF in their central regions.Our results suggest that the inner $\sim5$ kpc region of both galaxies, despite their different masses, formed at the same time and evolved with a similar star formation time-scale and chemical enrichment, confirming their early formation in the cluster build up. Only the overall metallicity and IMF radial profiles show differences connected with their different velocity dispersion profiles. The radial trend of the IMF positively correlates with both [ Z/H ] and $\sigma$ . While the trends of the IMF with metallicity agree with a global trend for both galaxies, the trends with the velocity dispersion exhibit differences. The outer regions show signs of mixed stellar populations with large differences in chemical content compared to the centers, but with similar old ages.

</div>

<div id="div_fig1">

<img src="tmp_2303.00044/./figures/ALF_ageZHimfALPHA_Hydra_FINAL.png" alt="Fig8" width="100%"/>

**Figure 8. -** \small{Age, metallicity and IMF slope trends across the Hydra I cluster center as retrieved with ALF. West is on the left, East is on the right. The bottom panel
shows the derived mismatch parameter $\alpha_r$.
Dash-dotted horizontal tan lines in the IMF and $\alpha_r$ panels outline the corresponding values for a Salpeter IMF, dashed those for a Kroupa IMF. Open diamonds refer to regions where the results are less robust due to lower S/N, velocity dispersion broadening and/or lack of important features. } (*fig:ageZimf*)

</div>
<div id="div_fig2">

<img src="tmp_2303.00044/./figures/Literature_comparison_KIN_PAPER.png" alt="Fig11" width="100%"/>

**Figure 11. -** \small{Velocity dispersion (upper panel) and radial velocity (lower panel) profiles as retrieved with ALF (orange).
Green triangles and red circles show an indication of the trends retrieved by the works of  ([Richtler, Salinas and Misgeld (2011)]())  and  ([Hilker, Richtler and Barbosa (2018)]()) , respectively, as they appear in figure $6$ of  ([Hilker, Richtler and Barbosa (2018)]())  for the case of position angle $108$◦.  Typical error bars are shown on the right of both panels.
} (*fig:kinematics*)

</div>
<div id="div_fig3">

<img src="tmp_2303.00044/./figures/ALF_ALLelements_Hydra_moreMask_FINALproc.png" alt="Fig9" width="100%"/>

**Figure 9. -** Similar to Figure \ref{fig:ageZimf but for all the retrieved elemental abundances.} (*fig:elements*)

</div>

In [8]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [10]:
export_markdown_summary(full_md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.00044.md
    + _build/html/tmp_2303.00044/./figures/ALF_ageZHimfALPHA_Hydra_FINAL.png
    + _build/html/tmp_2303.00044/./figures/Literature_comparison_KIN_PAPER.png
    + _build/html/tmp_2303.00044/./figures/ALF_ALLelements_Hydra_moreMask_FINALproc.png
